# DF

In [ ]:
import marimo as mo
import polars as pl

from join_tables import join

# @mo.persistent_cache
df = join()

taxon = pl.scan_csv(
    "gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True
)

# matching and contentious split
What makes a data point contentious is where it has duplicate speciesId.

In [ ]:
contentious = df.filter(
    (~pl.col("speciesId").is_first_distinct())
).with_columns(
    acceptedNameUsageID=pl.col("acceptedNameUsageID")
    .fill_null(pl.lit(-1))
    .cast(pl.Int64)
)

contentious.filter(
    ~pl.col("acceptedNameUsageID").is_in(pl.col("matched_taxonID").implode())
).sink_csv("contentious_part1.csv")

In [ ]:
matching = (df.filter((~pl.col("speciesId").is_duplicated()))).with_columns(
    acceptedNameUsageID=pl.col("acceptedNameUsageID")
    .fill_null(pl.lit(-1))
    .cast(pl.Int64)
)

contentious_part1 = pl.scan_csv("contentious_part1.csv")

unique_contentious = contentious_part1.filter(
    (~pl.col("speciesId").is_duplicated()) & (~pl.col("speciesId").is_null())
)  # added the non null filter, it is not in the R script.

unique_contentius_speciesId = (
    unique_contentious.select("speciesId").collect().to_series().implode()
)
contentious2 = contentious_part1.filter(
    ~pl.col("speciesId").is_in(unique_contentius_speciesId)
)
# contentious2.collect()["speciesId"]

matching = pl.concat(
    [matching, unique_contentious],
)

nomatch lazyframe is for BOS data point that has no current match in gbif data set.

In [ ]:
nomatch = (
    matching.filter(pl.col("matched_taxonID").is_null())
    .with_columns(
        taxonRank=pl.lit("BOSuncornirmedSpecies"),
        taxonomicStatus=pl.lit("BOSunformired"),
        parentNameUsageID=pl.lit(None),
    )
    .collect()
)

In [ ]:
accepted_taxon_lookup = (
    taxon.filter(pl.col("taxonomicStatus") == pl.lit("accepted"))
    .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    .filter(~pl.col("canonicalName").is_null())
    .filter(pl.col("canonicalName") != "")
    .sort("canonicalName")
    .filter((~pl.col("canonicalName").is_first_distinct()))
)

In [ ]:
_priority_columns = [
    "infraspecificEpithet",
    "specificEpithet",
    "genus",
    "family",
    "order",
    "class",
    "phylum",
    "kingdom",
    "domain",
]

# Unsure about this whole for loop block
for _c in _priority_columns:
    _a = accepted_taxon_lookup.filter(pl.col("canonicalName") == _c)["taxonID"]
    nomatch.with_columns(
        parentNameUsageID=pl.when(
            (pl.col("parentNameUsageID").is_nan())
            & (pl.col(_c).is_not_nan())
            & (pl.col(_c) != "")
        )
        .then(pl.lit(_a))
        .otherwise("parentNameUsageID")
    )

<span class="codehilite"><div class="highlight"><pre><span></span><span class="gt">Traceback (most recent call last):</span>
  File <span class="nb">&quot;/home/ryan/Documents/LKCNHM_volunteering/.venv/lib/python3.12/site-packages/marimo/_runtime/executor.py&quot;</span>, line <span class="m">138</span>, in <span class="n">execute_cell</span>
<span class="w">    </span><span class="n">exec</span><span class="p">(</span><span class="n">cell</span><span class="o">.</span><span class="n">body</span><span class="p">,</span> <span class="n">glbls</span><span class="p">)</span>
  File <span class="nb">&quot;/tmp/marimo_903543/__marimo__cell_nyaw_.py&quot;</span>, line <span class="m">15</span>, in <span class="n">&lt;module&gt;</span>
<span class="w">    </span><span class="n">_a</span> <span class="o">=</span> <span class="n">accepted_taxon_lookup</span><span class="o">.</span><span class="n">filter</span><span class="p">(</span><span class="n">pl</span><span class="o">.</span><span class="